
# 09 — Applied RAG Projects (ENT Guidelines + Contract Analyzer)  
RAG + simple Agents skeletons (ready for LangGraph/MCP extension).



## 1. ENT Guidelines RAG Assistant (Project Skeleton)



### Goal
- Load ENT guideline-like snippets
- Build a vector store
- Add a simple planner + retriever + answer agent flow


In [ ]:

!pip install langchain langchain-openai chromadb langchain-community --quiet

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

# ENT mini-corpus (simulated)
with open("ent_project_guidelines.txt", "w") as f:
    f.write("Chronic otitis media: First-line management includes assessment of hearing and infection control.\n")
    f.write("Acute sinusitis: Watchful waiting may be appropriate in mild cases, antibiotics reserved for severe or persistent symptoms.\n")
    f.write("Tonsillectomy: Indicated for recurrent tonsillitis meeting specific frequency criteria.\n")

loader_ent = TextLoader("ent_project_guidelines.txt")
ent_docs = loader_ent.load()

splitter_ent = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=30)
ent_chunks = splitter_ent.split_documents(ent_docs)

ent_emb = OpenAIEmbeddings()
ent_db = Chroma.from_documents(ent_chunks, ent_emb)

ent_llm = ChatOpenAI(model="gpt-4o-mini")


In [ ]:

# Simple planner / retriever / answer functions (agent-style skeleton)

def ent_planner(question: str) -> str:
    """Very simple planner: just returns the same question, but here is where you'd expand into a plan."""
    return f"Search ENT guideline for: {question}"

def ent_retriever(plan: str) -> str:
    docs = ent_db.similarity_search(plan, k=4)
    return "\n".join(d.page_content for d in docs)

def ent_answer_agent(question: str, context: str) -> str:
    prompt = f"""You are an ENT guideline assistant.
Use ONLY the context below. Do not give diagnosis or personalized/patient-specific advice.

Context:
{context}

Question: {question}

Answer in a neutral, guideline-style tone and clearly state that this is NOT medical advice.
"""
    return ent_llm.invoke(prompt).content

def ent_pipeline(question: str) -> str:
    plan = ent_planner(question)
    ctx = ent_retriever(plan)
    ans = ent_answer_agent(question, ctx)
    return ans

print(ent_pipeline("How is chronic otitis media managed in adults?"))



### MCP Tool Stubs (for extension)
Below is a placeholder for future MCP tools such as ICD-10 lookup or medical dictionary.


In [ ]:

# MCP-like tool stubs (no real MCP integration yet)

class ICD10Tool:
    def lookup(self, term: str) -> str:
        # Placeholder
        return f"[ICD10 lookup placeholder for term: {term}]"

class MedicalDictionaryTool:
    def search(self, term: str) -> str:
        # Placeholder
        return f"[Medical dictionary placeholder definition for: {term}]"

icd10_tool = ICD10Tool()
med_dict_tool = MedicalDictionaryTool()

print(icd10_tool.lookup("otitis media"))
print(med_dict_tool.search("sinusitis"))



## 2. Contract Analyzer Copilot (Project Skeleton)



### Goal
- Load contract-style clauses
- Build a vector store
- Provide simple clause Q&A and comparison skeletons


In [ ]:

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

with open("contracts_project_corpus.txt", "w") as f:
    f.write("Confidentiality: Each party agrees to keep all confidential information in strict confidence.\n")
    f.write("Limitation of liability: Liability shall be limited to direct damages and capped at the fees paid in the last 12 months.\n")
    f.write("Governing law: This Agreement shall be governed by the laws of the State of New York.\n")

contract_loader = TextLoader("contracts_project_corpus.txt")
contract_docs = contract_loader.load()

contract_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=30)
contract_chunks = contract_splitter.split_documents(contract_docs)

contract_emb = OpenAIEmbeddings()
contract_db = Chroma.from_documents(contract_chunks, contract_emb)

contract_llm = ChatOpenAI(model="gpt-4o-mini")


In [ ]:

def clause_detector(question: str) -> str:
    q_lower = question.lower()
    if "confidential" in q_lower:
        return "confidentiality"
    if "liability" in q_lower:
        return "limitation of liability"
    if "law" in q_lower or "jurisdiction" in q_lower:
        return "governing law"
    return "general"

def contract_retriever(clause_type: str) -> str:
    query = f"{clause_type} clause"
    docs = contract_db.similarity_search(query, k=3)
    return "\n".join(d.page_content for d in docs)

def contract_answer_agent(question: str, context: str) -> str:
    prompt = f"""You are a contract analysis assistant for legal professionals.
Use ONLY the context below.

Context:
{context}

Question: {question}

Provide a neutral summary of the relevant clause(s) and clearly say this is NOT legal advice.
"""
    return contract_llm.invoke(prompt).content

def contract_pipeline(question: str) -> str:
    clause_type = clause_detector(question)
    ctx = contract_retriever(clause_type)
    ans = contract_answer_agent(question, ctx)
    return ans

print(contract_pipeline("What does the limitation of liability clause say?"))



### MCP Tool Stubs (for extension)
Placeholders for tools like clause classifier, risk scoring, and template library.


In [ ]:

class ClauseClassifierTool:
    def predict(self, text: str) -> str:
        # Placeholder
        return "[Predicted clause type: limitation_of_liability]"

class RiskScoringTool:
    def evaluate(self, clause: str) -> str:
        # Placeholder
        return "[Risk score placeholder for clause]"

class TemplateLibraryTool:
    def fetch_template(self, clause_type: str) -> str:
        # Placeholder
        return f"[Template clause for type: {clause_type}]"

clause_tool = ClauseClassifierTool()
risk_tool = RiskScoringTool()
template_tool = TemplateLibraryTool()

print(clause_tool.predict("Liability is capped at fees paid."))
print(risk_tool.evaluate("Liability is uncapped."))
print(template_tool.fetch_template("limitation_of_liability"))



## End of Notebook 09  
ENT Guidelines Assistant and Contract Analyzer Copilot skeletons are ready.
